In [13]:

parameter= {
    "model": "microsoft/deberta-v3-base",
    "max_length": 512,
    "batch_size": 8,
    "lr": 1e-5,
    "filter_no_pii_percent_allow": 0.2,
}


In [14]:

import json
import numpy as np

data = json.load(open("data/train.json"))

print(len(data))
print(data[0].keys())

x = data[0]

print(x["tokens"][:10])
print(x["labels"][:10])
print(x["trailing_whitespace"][:10])

6807
dict_keys(['document', 'full_text', 'tokens', 'trailing_whitespace', 'labels'])
['Design', 'Thinking', 'for', 'innovation', 'reflexion', '-', 'Avril', '2021', '-', 'Nathalie']
['O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'B-NAME_STUDENT']
[True, True, True, True, False, False, True, False, False, True]


In [15]:
x.keys()

dict_keys(['document', 'full_text', 'tokens', 'trailing_whitespace', 'labels'])

In [16]:

from itertools import chain

all_labels = sorted(list(set(chain(*[x["labels"] for x in data]))))
label2id = {l: i for i,l in enumerate(all_labels)}
id2label = {v:k for k,v in label2id.items()}

id2label

{0: 'B-EMAIL',
 1: 'B-ID_NUM',
 2: 'B-NAME_STUDENT',
 3: 'B-PHONE_NUM',
 4: 'B-STREET_ADDRESS',
 5: 'B-URL_PERSONAL',
 6: 'B-USERNAME',
 7: 'I-ID_NUM',
 8: 'I-NAME_STUDENT',
 9: 'I-PHONE_NUM',
 10: 'I-STREET_ADDRESS',
 11: 'I-URL_PERSONAL',
 12: 'O'}

In [17]:
import random

def tokenize(example, tokenizer, label2id, max_length):
    text = []
    labels = []
    for t, l, ws in zip(example["tokens"], example["labels"], example["trailing_whitespace"]):
        text.append(t)
        labels.extend([l]*len(t))
        if ws:
            text.append(" ")
            labels.append("O")
    
    tokenized = tokenizer("".join(text), return_offsets_mapping=True, max_length=max_length, truncation=True, padding="max_length")
    labels = np.array(labels)
    text = "".join(text)
    token_labels = []
    
    for start_idx, end_idx in tokenized.offset_mapping:
        # CLS token
        if start_idx == 0 and end_idx == 0: 
            token_labels.append(label2id["O"])
            continue
        
        # case when token starts with whitespace
        if text[start_idx].isspace():
            start_idx += 1
        
        while start_idx >= len(labels):
            start_idx -= 1
            
        token_labels.append(label2id[labels[start_idx]])
        
    length = len(tokenized.input_ids)
    return {
        **tokenized,
        "labels": token_labels,
        "length": length
    }

# https://www.kaggle.com/competitions/pii-detection-removal-from-educational-data/discussion/468844
def filter_no_pii(example, percent_allow=parameter["filter_no_pii_percent_allow"]):
    # Return True if there is PII
    # Or 20% of the time if there isn't
    has_pii = set("O") != set(example["labels"])
    return has_pii or (random.random() < percent_allow)
    



In [18]:
import torch
import json

class PiiDataset(torch.utils.data.Dataset):
    def __init__(self, dataset, tokenizer, label2id, max_length):
        self.dataset = dataset
        self.tokenizer = tokenizer
        self.label2id = label2id
        self.max_length = max_length
        
    def __getitem__(self, idx):
        vals=tokenize(self.dataset[idx], self.tokenizer, self.label2id, self.max_length)

        input_ids = torch.tensor(vals["input_ids"])
        attention_mask = torch.tensor(vals["attention_mask"])
        labels = torch.tensor(vals["labels"], dtype=torch.long)

        return input_ids, attention_mask, labels
    
    def __len__(self):
        return len(self.dataset)
    

data = json.load(open("data/train.json"))


from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained(parameter["model"])

my_dataset=PiiDataset(data, tokenizer, label2id, parameter["max_length"])


Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
c:\Users\Shadow\anaconda3\envs\mytorch\lib\site-packages\transformers\convert_slow_tokenizer.py:470: UserWarning: The sentencepiece tokenizer that you are converting to a fast tokenizer uses the byte fallback option which is not implemented in the fast tokenizers. In practice this means that the fast version of the tokenizer can produce unknown tokens whereas the sentencepiece version would have converted these unknown tokens into a sequence of byte tokens matching the original piece of text.
  warnings.warn(
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [19]:

loader=torch.utils.data.DataLoader(my_dataset, batch_size=8, shuffle=True)

for id, attention_mask, labels in loader:
    print(id.shape)
    print(attention_mask.shape)
    print(labels.shape)
    break

torch.Size([8, 512])
torch.Size([8, 512])
torch.Size([8, 512])


In [20]:
from transformers import AutoModelForTokenClassification

device=torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
class MyModel(torch.nn.Module):
    def __init__(self, model_name, num_labels):
        super().__init__()
        self.model = AutoModelForTokenClassification.from_pretrained(model_name, num_labels=num_labels)
        self.softmax=torch.nn.Softmax(dim=-1)
        
    def forward(self, input_ids, attention_mask, labels):
        out=self.model(input_ids, attention_mask=attention_mask, labels=labels)['logits']
        out=self.softmax(out)
        return out


model = MyModel(parameter['model'], len(label2id))
model= model.to(device)
for id, attention_mask, labels in loader:
    print(id.shape)
    print(attention_mask.shape)
    print(labels.shape)
    print(labels)
    id = id.to(device)
    attention_mask = attention_mask.to(device)
    labels = labels.to(device)
    print(model(id, attention_mask, labels).shape)
    break

#free gpu memory
del model
torch.cuda.empty_cache()



Some weights of DebertaV2ForTokenClassification were not initialized from the model checkpoint at microsoft/deberta-v3-base and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


torch.Size([8, 512])
torch.Size([8, 512])
torch.Size([8, 512])
tensor([[12, 12, 12,  ..., 12, 12, 12],
        [12, 12, 12,  ..., 12, 12, 12],
        [12, 12, 12,  ..., 12, 12, 12],
        ...,
        [12, 12, 12,  ..., 12, 12, 12],
        [12, 12, 12,  ..., 12, 12, 12],
        [12, 12, 12,  ..., 12, 12, 12]])
torch.Size([8, 512, 13])


In [21]:
import neptune
from tqdm.notebook import tqdm

class Learner():
    def __init__(self, model, train_dataloader, valid_dataloader, parameter=None):
        self.model=model
        #self.loss_fn=torch.nn.CrossEntropyLoss()
        self.loss_fn=torch.nn.CrossEntropyLoss(ignore_index=-100)
        self.device=torch.device("cpu")
        if torch.cuda.is_available():
            self.device=torch.device("cuda")
        #elif torch.backends.mps.is_available():
        #    self.device=torch.device("mps")

        self.model.to(self.device)
        self.run = neptune.init_run(
            project="bernd.heidemann/PII",
            api_token="eyJhcGlfYWRkcmVzcyI6Imh0dHBzOi8vYXBwLm5lcHR1bmUuYWkiLCJhcGlfdXJsIjoiaHR0cHM6Ly9hcHAubmVwdHVuZS5haSIsImFwaV9rZXkiOiIzNjBlYzVkNi0zZTUwLTQ1ODYtODhlNC02NDUxNDg0MDdjNzUifQ==",
        )  # your credentials
        self.parameter = parameter
        self.train_dataloader = train_dataloader
        self.valid_dataloader = valid_dataloader
        self.run["parameters"] = {
            **self.parameter
        }

    def fit(self, lr=0.001, epochs=10):
        optimizer=torch.optim.AdamW(self.model.parameters(), lr=lr)
        scheduler=scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=epochs)
        bar = tqdm(total=len(self.train_dataloader) * epochs, desc="Training")
        bar.set_description("Epoch 0/{}".format(epochs))

        for epoch in range(epochs):
            self.model.train()            
            for ids, att_mask, labels in self.train_dataloader:
                
                ids=ids.to(self.device)
                labels=labels.to(self.device)
                att_mask=att_mask.to(self.device)
                pred=self.model(ids, att_mask, labels)
                # reshape pred to [batch_size, num_classes, sequence_length]
                metrics=self.f_beta_score_multiclass(labels, pred)
                self.run["train_f_beta_score"].log(metrics["f_beta"])
                self.run["train_precision"].log(metrics["precision"])
                self.run["train_recall"].log(metrics["recall"])
                self.run["train_true_positives"].log(metrics["true_positives"])
                self.run["train_false_positives"].log(metrics["false_positives"])
                self.run["train_false_negatives"].log(metrics["false_negatives"])
                pred = pred.permute(0, 2, 1)
                loss=self.loss_fn(pred, labels)
                self.run["train_loss"].log(loss.item())
                loss.backward()
                optimizer.step()
                optimizer.zero_grad()
                bar.update(1)
            scheduler.step()
            self.model.eval()
            # log current state to neptune
            if self.valid_dataloader is not None:
                metrics=self.get_accuracy()
                self.run["valid_accuracy"].log(metrics["accuracy"])
                self.run["valid_loss"].log(metrics["loss"])
                self.run["valid_f_beta_score"].log(metrics["f_beta_score"])
                self.run["valid_precision"].log(metrics["precision"])
                self.run["valid_recall"].log(metrics["recall"])
                self.run["valid_true_positives"].log(metrics["true_positives"])
                self.run["valid_false_positives"].log(metrics["false_positives"])
                self.run["valid_false_negatives"].log(metrics["false_negatives"])
                bar.set_description("Epoch {}/{} validAccuracy: {:.2f} validLoss: {:.2f}".format(epoch+1, epochs, metrics["accuracy"], metrics["loss"]))
            
                
    def get_accuracy(self):
        self.model.eval()
        with torch.no_grad():
            correct=0
            losses=[]
            batch_metrics=[]
            for ids, att_mask, labels in self.valid_dataloader:
                ids=ids.to(self.device)
                labels=labels.to(self.device)
                att_mask=att_mask.to(self.device)
                pred=self.model(ids, att_mask, labels)
                f_beta_score_results = self.f_beta_score_multiclass(labels, pred)
                batch_metrics.append(f_beta_score_results)
                pred = pred.permute(0, 2, 1)
                loss=self.loss_fn(pred, labels)
                losses.append(loss.item())
                pred=torch.argmax(pred, dim=1)
                correct+=torch.sum(pred==labels).item()
            # calc mean of the dict entries in batch_metrics
            f_beta_scores = np.mean([x["f_beta"] for x in batch_metrics])
            precision = np.mean([x["precision"] for x in batch_metrics])
            recall = np.mean([x["recall"] for x in batch_metrics])
            true_positives = np.mean([x["true_positives"] for x in batch_metrics])
            false_positives = np.mean([x["false_positives"] for x in batch_metrics])
            false_negatives = np.mean([x["false_negatives"] for x in batch_metrics])

            return {
                "accuracy": correct/len(self.valid_dataloader.dataset),
                "loss": np.mean(losses),
                "f_beta_score": f_beta_scores,
                "precision": precision,
                "recall": recall,
                "true_positives": true_positives,
                "false_positives": false_positives,
                "false_negatives": false_negatives
            }
        
    def f_beta_score_multiclass(self, y_true, y_pred, beta=5, epsilon=1e-7):

        # assert y_pred has values between 0 and 1
        assert y_pred.min() >= 0
        assert y_pred.max() <= 1

        y_true_one_hot = torch.nn.functional.one_hot(y_true, num_classes=y_pred.shape[2])
    
        # Berechnung von True Positives, False Positives und False Negatives
        tp = torch.sum(y_true_one_hot * y_pred, dim=0)
        fp = torch.sum((1 - y_true_one_hot) * y_pred, dim=0)
        fn = torch.sum(y_true_one_hot * (1 - y_pred), dim=0)

        # Summierung über alle Klassen
        tp_sum = torch.sum(tp)
        fp_sum = torch.sum(fp)
        fn_sum = torch.sum(fn)

        # Berechnung von Präzision und Recall
        precision = tp_sum / (tp_sum + fp_sum + epsilon)
        recall = tp_sum / (tp_sum + fn_sum + epsilon)

        # Berechnung des F-Beta-Scores
        f_beta = (1 + beta**2) * (precision * recall) / ((beta**2 * precision) + recall + epsilon)

        return {
            "f_beta": f_beta.item(),
            "precision": precision.item(),
            "recall": recall.item(),
            "true_positives": tp_sum.item(),
            "false_positives": fp_sum.item(),
            "false_negatives": fn_sum.item()
        }

    

In [22]:
data = json.load(open("data/train.json"))

print("original len",  len(data))
# this increased the 
data_filterd = list(filter(filter_no_pii, data))
print("filtered len", len(data_filterd))

data_len=len(data_filterd)

train_len=int(len(data_filterd)*0.8)
valid_len=len(data_filterd)-train_len

train_data_idx=np.random.choice(data_len, train_len, replace=False)
valid_data_idx=np.array(list(set(range(data_len))-set(train_data_idx)))

train_data=[data_filterd[i] for i in train_data_idx]
valid_data=[data_filterd[i] for i in valid_data_idx]

print("len train ds", len(train_data))
print("len valid ds", len(valid_data))

original len 6807
filtered len 2071
len train ds 1656
len valid ds 415


In [23]:
len(label2id)

13

In [24]:
from tokenizers import AddedToken

# set environment variables: TOKENIZERS_PARALLELISM=false
import os
os.environ["TOKENIZERS_PARALLELISM"] = "false"

tokenizer = AutoTokenizer.from_pretrained(parameter["model"])
tokenizer.add_tokens(AddedToken("\n", normalized=False))

train_dataset = PiiDataset(train_data, tokenizer, label2id, parameter["max_length"])
valid_dataset = PiiDataset(valid_data, tokenizer, label2id, parameter["max_length"])
train_dataloader = torch.utils.data.DataLoader(train_dataset, batch_size=parameter['batch_size'], shuffle=True)
valid_dataloader = torch.utils.data.DataLoader(valid_dataset, batch_size=parameter['batch_size'], shuffle=False)
my_model=MyModel(parameter['model'], len(label2id))

learner=Learner(my_model, train_dataloader, valid_dataloader, parameter=parameter)
learner.fit(lr=parameter['lr'], epochs=2)

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
Some weights of DebertaV2ForTokenClassification were not initialized from the model checkpoint at microsoft/deberta-v3-base and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


https://app.neptune.ai/bernd.heidemann/PII/e/PII-27


Training:   0%|          | 0/414 [00:00<?, ?it/s]

In [25]:
for id, attention_mask, labels in valid_dataloader:
    id=id.to(device)
    attention_mask=attention_mask.to(device)
    labels=labels.to(device)
    preds=learner.model(id, attention_mask, labels).argmax(dim=2)
    # print frequency of labels
    print("predictions:" , torch.bincount(preds.flatten()))
    print("ground truth: ", torch.bincount(labels.flatten()))
    break

predictions: tensor([   0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
        4096], device='cuda:0')
ground truth:  tensor([   0,    0,    7,    0,    0,    0,    0,    0,    6,    0,    0,    0,
        4083], device='cuda:0')


In [26]:
learner.fit(lr=parameter['lr'], epochs=20)

Training:   0%|          | 0/4140 [00:00<?, ?it/s]

In [27]:
# save model

torch.save(learner.model.state_dict(), "model.pt")